# <center> Tracer la courbe ROC et test pour la Learning Curve </center>

In [ ]:
from bigml.api import BigML
from pandas import read_csv
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
api = BigML(project= 'project/5d94a407eba31d45c8000088')

In [ ]:
df = read_csv('../handson-ml2/kaggle-give-me-credit-Ensemble_confusion.csv')
df

In [ ]:
data = read_csv('../handson-ml2/kaggle-give-me-credit-Deepenet_confusion.csv')
data

## Tracer la ROC

In [ ]:
import time
#Timer le code
start = time.time()
#False positive rate
fpr = []
# True positive rate
tpr = []

thresh = [(n/100) for n in range(0,100)]

count = df['error'].value_counts()
number_of_positive = count[2] + count[1]
number_of_negative = count[3] + count[0]

for t in thresh:
    FP = 0
    TP = 0
    for i in range(len(df['1 probability'])):
        if (df['1 probability'][i]>t):
            if df['SeriousDlqin2yrs'][i] == 1:
                TP = TP + 1
            if df['SeriousDlqin2yrs'][i] == 0:
                FP = FP + 1
    fpr.append(FP/float(number_of_negative))
    tpr.append(TP/float(number_of_positive))
stop = time.time()
print(stop-start)
plt.plot(fpr,tpr)
plt.ylabel("True positive rate")
plt.xlabel("False positive rate")
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.show()

## Test pour la learning Curve

In [ ]:
X = []
Y = []

m = 0
n = 0

for i in range(0,100): 
    # On crée le nouveau dataset de train à partir du dataset de train initial
    df_split = df.head(int(len(df)*((i+10)/100)))
    positif = df_split.loc[df_split['SeriousDlqin2yrs'] == 1] 
    negatif = df_split.loc[df_split['SeriousDlqin2yrs'] == 0]
    for threshold_pos in positif['1 probability']:
        for threshold_neg in negatif['1 probability']:
            if threshold_pos > threshold_neg:
                m += 1 
            n += 1
            AUC = round((m/n)*100,2)
    # On stocke dans une liste la valeur de l'axe des abscisses et la valeur de l'AUC à partir de la donnée BigML
    X.append(i)
    Y.append(AUC)
    

fig, ax = plt.subplots()
ax.plot(X, Y)
ax.set(xlabel='Taille du dataset train en %', ylabel='AUC',  title='Learning Curve (AUC / % Split)')
plt.show()

In [ ]:
X_deepnet = []
Y_deepnet = []

m_deepnet = 0
n_deepnet = 0

for i_deepnet in range(0,100): 
    # On crée le nouveau dataset de train à partir du dataset de train initial
    data_split = data.head(int(len(data)*((i_deepnet+10)/100)))
    positif = data_split.loc[data_split['SeriousDlqin2yrs'] == 1] 
    negatif = data_split.loc[data_split['SeriousDlqin2yrs'] == 0]
    for threshold_pos in positif['1 probability']:
        for threshold_neg in negatif['1 probability']:
            if threshold_pos > threshold_neg:
                m_deepnet += 1 
            n_deepnet += 1
            AUC_deepnet = round((m_deepnet/n_deepnet)*100,2)
    # On stocke dans une liste la valeur de l'axe des abscisses et la valeur de l'AUC à partir de la donnée BigML
    X_deepnet.append(i_deepnet)
    Y_deepnet.append(AUC_deepnet)
    

fig, ax = plt.subplots()
ax.plot(X_deepnet, Y_deepnet)
ax.set(xlabel='Taille du dataset Train', ylabel='AUC',  title='Performance Curve (AUC / % Split)')
plt.show()

In [ ]:
plt.plot(X, Y, color='darkblue')
plt.text(80,86,'Ensemble', color='darkblue')
plt.ylabel('AUC')
plt.xlabel('Taille du dataset train en %')
plt.plot(X_deepnet, Y_deepnet, color='red')
plt.text(80,83.5,'Deepnet', color='red')
plt.title('Performance Curve')
plt.show()
plt.savefig("Courbe de Performance.png")